In [1]:
import torch
from model.ssd import *
from utils import *
from data import *
from torch.optim import SGD
from train import train
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

In [2]:
TOTAL_PRIORS_NUM = 9128
TOTAL_NUM_OF_CLASSES = 91

In [3]:
if torch.cuda.is_available():
#     device = torch.device('cuda:0')
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [4]:
dataset_train_path = "/home/sarieh/coco_dataset"
dataset_val_path = "/home/sarieh/coco_dataset"
annot_train_path = "/home/sarieh/coco_dataset/annotations"

In [5]:
train_data = CocoData(dataset_train_path, annot_train_path, 
                      transform=CustomCocoTransform,
                      device = device,
                      labels_path = './data/coco/annotations/labels.txt')

loading annotations into memory...
Done (t=11.24s)
creating index...
index created!


In [6]:
val_data = CocoData(dataset_train_path, annot_train_path,
                    transform=CustomCocoTransform,
                    device = device,
                    split="validation",
                    labels_path = './data/coco/annotations/labels.txt')

loading annotations into memory...
Done (t=2.00s)
creating index...
index created!


In [7]:
criterion = MultiboxLoss(TOTAL_PRIORS_NUM, TOTAL_NUM_OF_CLASSES,threshold=0.5, alpha=1)

In [8]:
epochs = 1

In [9]:
priors = [4, 7, 6, 7, 5, 5]
channels_in = [128, 256, 512, 256, 256, 256]
loc_channel_out = [4 * prior for prior in priors]
pred_channel_out = [NUM_OF_CLASSES * prior for prior in priors]
base_model = models.resnet18(pretrained=True)
features = FeatureNet(base_model)
    
for param in base_model.parameters():
    param.requires_grad=False

In [10]:
model = PredConvNet(channels_in, loc_channel_out, pred_channel_out, features.features)

In [11]:
optimizer = SGD(model.parameters(), lr=2.5e-2, momentum=0.9)

In [17]:
train_loader = DataLoader(dataset=train_data, collate_fn = train_data.collate_fn ,batch_size=22)
validation_loader = DataLoader(dataset=val_data, collate_fn = val_data.collate_fn , batch_size=16)

In [18]:
loss = train(
    model = model,
    train_loader = train_loader,
    validation_loader = validation_loader,
    optimizer = optimizer,
    criterion = criterion,
    epochs = epochs,
    device = device
)

RuntimeError: CUDA out of memory. Tried to allocate 122.00 MiB (GPU 0; 5.80 GiB total capacity; 429.79 MiB already allocated; 121.00 MiB free; 746.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF